# FitFirst Study

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
from pyvis.network import Network
from coloring.fit_first import fit_first
from utils import generate_k_colorable_graph

### Specify parameters here

In [2]:
N = 50    # Number of online k-colorable graphs to generate at each step
k_range = [2,3,4]
n_range = range(100,3001,100)

In [3]:
def convert_to_dataframe(data:list) -> pd.DataFrame:
    df = pd.DataFrame(data, columns=['k', 'n', 'colors_used', 'ratio', 'N'])
    return df

In [6]:
# <k, n, colors_used, ratio, N>
data = []
for k in k_range:
    for n in tqdm(n_range):
        for _ in range(N):
            # Initialize the graph
            G = generate_k_colorable_graph(k, n, 0.8)

            # Simulate online coloring
            for idx in range(2, n+1):
                subgraph = G.subgraph(range(idx))
                colors = fit_first(subgraph)
                # Color the graph
                for node, color in colors.items():
                    G.nodes[node]['group'] = color
                
                # Determine the number of colors used
                colors_used = len(set(colors.values()))

                # Determine the ratio of colors used
                ratio = colors_used / k

                # Append the data
                data.append([k, n, colors_used, ratio, N])
        # Save checkpoint
        _df = convert_to_dataframe(data)
        _df.to_feather(f'../data/fit_first_checkpoint.ftr')

# Save the data
df = convert_to_dataframe(data)
df.to_feather(f'../data/fit_first.ftr')

 13%|█▎        | 2/15 [00:38<04:07, 19.03s/it]


KeyboardInterrupt: 